In [ ]:
pip install tensorflow


In [10]:
import pandas as pd
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import MinMaxScaler
from difflib import get_close_matches


In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
import numpy as np


2024-11-10 13:31:09.391929: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
import matplotlib.pyplot as plt

In [16]:
# Carregar os datasets
cervejas = pd.read_csv('./datasets/preprocessed/CERVEJAS_processed.csv')
harmonizacoes = pd.read_csv('./datasets/preprocessed/HARMONIZACOES_processed.csv')
ingredientes = pd.read_csv('./datasets/preprocessed/INGREDIENTES_normalizados_processed.csv')
pratos = pd.read_csv('./datasets/preprocessed/PRATOS_processed.csv')
treino_final = pd.read_csv('./datasets/preprocessed/PRATOxCERVEJA_processed.csv')

In [165]:
treino_final

,estilos_cervejas_brejas,pratos_pt,pratos,estilos_cervejas,category,estilo,ibu,og,fg,abv,...,sandalwood,green tea,patchouli,tangy,pleasant,cardboard,maple syrup,skunky,tomato leaf,outdoor
0,american pale ale,"{'atum (massas ou sanduíches)', 'enchiladas (t...",Hamburger,American Pale Ale,Pale American Ale,American Pale Ale,0.3,0.182692,0.40,0.181034,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,american pale ale,"{'atum (massas ou sanduíches)', 'enchiladas (t...",Hamburger,American Pale Ale,Pale American Ale,American Pale Ale,0.3,0.182692,0.40,0.181034,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,american pale ale,"{'atum (massas ou sanduíches)', 'enchiladas (t...",Hamburger,American Pale Ale,Pale American Ale,American Pale Ale,0.3,0.182692,0.40,0.181034,...,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,1.0,0.0
3,american pale ale,"{'atum (massas ou sanduíches)', 'enchiladas (t...",Hamburger,American Pale Ale,Pale American Ale,American Pale Ale,0.3,0.182692,0.40,0.181034,...,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,1.0,0.0
4,american pale ale,"{'atum (massas ou sanduíches)', 'enchiladas (t...",Hamburger,American Pale Ale,Pale American Ale,American Pale Ale,0.3,0.182692,0.40,0.181034,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55399,belgian pale ale,"{'lasanha', 'frango assado', 'hummus (patê de ...",Lasagna,Belgian Pale Ale,Belgian Ale,Belgian Pale Ale,0.3,0.269231,0.48,0.237069,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
55400,belgian pale ale,"{'lasanha', 'frango assado', 'hummus (patê de ...",Lasagna,Belgian Pale Ale,Belgian Ale,Belgian Pale Ale,0.3,0.269231,0.48,0.237069,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
55401,belgian pale ale,"{'lasanha', 'frango assado', 'hummus (patê de ...",Lasagna,Belgian Pale Ale,Belgian Ale,Belgian Pale Ale,0.3,0.269231,0.48,0.267241,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
55402,belgian pale ale,"{'lasanha', 'frango assado', 'hummus (patê de ...",Lasagna,Belgian Pale Ale,Belgian Ale,Belgian Pale Ale,0.3,0.269231,0.48,0.267241,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# atributos de cerveja
range_atributos_cerveja = ['ibumin', 'ibumax', 'ogmin', 'ogmax', 'fgmin', 'fgmax', 'abvmin', 'abvmax', 'srmmin', 'srmmax']
atributos_cerveja = ['ibu', 'og', 'fg', 'abv', 'srm']

# atributos de sabor
todos_sabores = [col for col in ingredientes.columns if col not in ['ingrediente', 'categoria']]


In [156]:
# treino_final.to_csv('./datasets/preprocessed/teste.csv', index=False)

In [ ]:
### DEFININDO VARIAVEIS ###

In [20]:
# Definindo as colunas X e Y
X = treino_final[atributos_cerveja]  # atributos de sabor da cerveja
Y = treino_final[todos_sabores]  # atributos de sabor dos pratos



In [171]:
Y

,nutty,peanut,cocoa,musty,roasted,hazelnut,roast,earthy,potato,woody,...,sandalwood,green tea,patchouli,tangy,pleasant,cardboard,maple syrup,skunky,tomato leaf,outdoor
0,0.17,0.060606,0.152174,0.157143,0.115385,0.368421,0.230769,0.187500,0.133333,0.092166,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.18,0.090909,0.130435,0.200000,0.153846,0.368421,0.230769,0.187500,0.166667,0.216590,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.43,0.363636,0.434783,0.342857,0.269231,0.473684,0.230769,0.416667,0.466667,0.221198,...,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,1.0,0.0
3,0.64,0.575758,0.652174,0.557143,0.538462,0.842105,0.692308,0.635417,0.600000,0.322581,...,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,1.0,0.0
4,0.17,0.060606,0.152174,0.157143,0.115385,0.368421,0.230769,0.187500,0.133333,0.092166,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55399,0.36,0.363636,0.456522,0.400000,0.192308,0.315789,0.076923,0.395833,0.333333,0.548387,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
55400,0.36,0.363636,0.456522,0.400000,0.192308,0.315789,0.076923,0.395833,0.333333,0.548387,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
55401,0.36,0.363636,0.456522,0.400000,0.192308,0.315789,0.076923,0.395833,0.333333,0.548387,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
55402,0.36,0.363636,0.456522,0.400000,0.192308,0.315789,0.076923,0.395833,0.333333,0.548387,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# Dividindo os dados em conjunto de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



In [ ]:
### TREINAMENTO DO MODELO ###

In [24]:
# Construindo a rede neural
modelo_nn = Sequential()
modelo_nn.add(Input(shape=(len(atributos_cerveja),)))  # Definindo a forma da entrada usando Input
modelo_nn.add(Dense(256, activation='relu'))  # Primeira camada oculta
modelo_nn.add(Dense(128, activation='relu'))  # Segunda camada oculta
modelo_nn.add(Dense(64, activation='relu'))  # Terceira camada oculta
modelo_nn.add(Dense(len(todos_sabores), activation='linear'))  # Camada de saída com atributos de sabor de comidas


In [26]:
# Compilando o modelo
modelo_nn.compile(optimizer='adam', loss='mean_squared_error')



In [28]:
# Treinando o modelo
history = modelo_nn.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_test, Y_test))



Epoch 1/100
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0630 - val_loss: 0.0548
Epoch 2/100
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0551 - val_loss: 0.0540
Epoch 3/100
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0538 - val_loss: 0.0538
Epoch 4/100
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0535 - val_loss: 0.0532
Epoch 5/100
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0533 - val_loss: 0.0529
Epoch 6/100
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0527 - val_loss: 0.0526
Epoch 7/100
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0526 - val_loss: 0.0525
Epoch 8/100
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0523 - val_loss: 0.0523
Epoch 9/100
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0521 - val_loss: 0.0526
Epoch 10/100
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0519 - val_loss: 0.0520
Epoch 11/100
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0523 - val_loss: 0.0522
Epoch 12/100
1386/1

In [30]:
# Avaliando o modelo
score = modelo_nn.evaluate(X_test, Y_test, verbose=0)
print(f"Mean Squared Error no conjunto de teste: {score}")


Mean Squared Error no conjunto de teste: 0.04881726950407028


In [ ]:
### VISUALIZANDO OS RESULTADOS ###

In [ ]:
### FUNÇÕES PARA PREVER RECOMENDACOES ###

In [119]:
# # similaridade min-max (com base na maxima distancia e minima distancia)

# def encontrar_pratos_por_cerveja(nome_cerveja, cervejas_df, pratos_df, modelo, atributos_cerveja, todos_sabores):
#     # Converter o nome da cerveja para minúsculas
#     nome_cerveza_lower = nome_cerveja.lower()
    
#     # Buscar os atributos da cerveja na coluna 'estilo' ignorando maiúsculas e minúsculas
#     cerveja = cervejas_df[cervejas_df['estilo'].str.lower().str.contains(nome_cerveza_lower, na=False)]
    
#     if cerveja.empty:
#         # Encontrar estilos similares se não encontrar a cerveja
#         estilos_similares = set(get_close_matches(nome_cerveja, cervejas_df['estilo'].tolist(), n=3, cutoff=0.5))
#         if estilos_similares:
#             estilo_similar = list(estilos_similares)[0]  # Pega o primeiro estilo similar encontrado
#             mensagem = f"Cerveja similar encontrada: '{estilo_similar}'."
            
#             # Buscar os atributos da primeira cerveja similar
#             cerveja_similar = cervejas_df[cervejas_df['estilo'].str.lower() == estilo_similar.lower()]
#             atributos_cerveja_valores = cerveja_similar[atributos_cerveja].values
#         else:
#             return f"Cerveja '{nome_cerveja}' não encontrada e nenhum estilo similar disponível.", []
#     else:
#         # Se a cerveja foi encontrada `exatamente`
#         mensagem = f"Cerveja encontrada: '{nome_cerveja}'."
#         atributos_cerveja_valores = cerveja[atributos_cerveja].values
    
#     # Prever os atributos dos pratos usando o modelo
#     atributos_previstos = modelo.predict(atributos_cerveja_valores)
    
#     # Verifica se os atributos previstos estão na forma correta
#     if atributos_previstos.shape[0] == 0:
#         return mensagem, "Não foi possível prever os atributos para a cerveja."

#     # Calcular a média dos atributos previstos (agregacao)
#     atributos_previstos_medios = np.mean(atributos_previstos, axis=0).reshape(1, -1)

#     # Extrair os atributos dos pratos usando a lista todos_sabores
#     atributos_pratos = pratos_df[todos_sabores].values
    
#     # Verificar se o DataFrame de pratos está vazio
#     if atributos_pratos.size == 0:
#         return mensagem, "A base de pratos está vazia."

#     # Calcular a distância entre os atributos previstos (média) e os atributos dos pratos
#     distancias = pairwise_distances(atributos_pratos, atributos_previstos_medios)

#     # Obter os índices dos 5 pratos mais próximos
#     indices_mais_proximos = np.argsort(distancias.flatten())[:5]

#     # Preparar a lista de pratos e as distâncias correspondentes
#     pratos_encontrados = []
#     for indice in indices_mais_proximos:
#         prato = pratos_df.iloc[indice]['pratos_allrecipes']
#         distancia = distancias[indice][0]
#         # Calcular a porcentagem inversa da distância
#         porcentagem = 100 - ((distancia / max(distancias.flatten())) * 100)
#         pratos_encontrados.append((prato, porcentagem))

#     # Retornar a mensagem e a lista de pratos encontrados
#     return mensagem, pratos_encontrados if pratos_encontrados else "Nenhum prato encontrado."


In [131]:

def encontrar_pratos_por_cerveja(nome_cerveja, cervejas_df, pratos_df, modelo, atributos_cerveja, todos_sabores):
    # Converter o nome da cerveja para minúsculas
    nome_cerveja_lower = nome_cerveja.lower()
    
    # Buscar os atributos da cerveja na coluna 'estilo' ignorando maiúsculas e minúsculas
    cerveja = cervejas_df[cervejas_df['estilo'].str.lower().str.contains(nome_cerveja_lower, na=False)]
    
    if cerveja.empty:
        estilos_similares = set(get_close_matches(nome_cerveja, cervejas_df['estilo'].tolist(), n=3, cutoff=0.5))
        if estilos_similares:
            estilo_similar = list(estilos_similares)[0]
            mensagem = f"Cerveja similar encontrada: '{estilo_similar}'."
            cerveja_similar = cervejas_df[cervejas_df['estilo'].str.lower() == estilo_similar.lower()]
            atributos_cerveja_valores = cerveja_similar[atributos_cerveja].values
        else:
            return f"Cerveja '{nome_cerveja}' não encontrada e nenhum estilo similar disponível.", []
    else:
        mensagem = f"Cerveja encontrada: '{nome_cerveja}'."
        atributos_cerveja_valores = cerveja[atributos_cerveja].values
    
    # Prever os atributos dos pratos usando o modelo
    atributos_previstos = modelo.predict(atributos_cerveja_valores)
    
    if atributos_previstos.shape[0] == 0:
        return mensagem, "Não foi possível prever os atributos para a cerveja."

    atributos_previstos_medios = np.mean(atributos_previstos, axis=0).reshape(1, -1)
    atributos_pratos = pratos_df[todos_sabores].values
    
    if atributos_pratos.size == 0:
        return mensagem, "A base de pratos está vazia."

    pratos_encontrados = []

    # Comparar os atributos previstos com os atributos dos pratos
    for i, atributos_prato in enumerate(atributos_pratos):
        similaridades_individuais = []

        for j, (previsto, encontrado) in enumerate(zip(atributos_previstos_medios[0], atributos_prato)):
            # Calculando a similaridade para o sabor j, baseado na diferença entre os valores
            similaridade = 100 * (1 - abs(previsto - encontrado))  # A similaridade vai de 0 a 100
            similaridades_individuais.append(similaridade)
        
        # Média das similaridades
        similaridade_media = np.mean(similaridades_individuais)

        prato = pratos_df.iloc[i]['pratos_allrecipes']
        pratos_encontrados.append((prato, similaridade_media))

    # Ordenar os pratos pelo valor mais alto de similaridade média
    pratos_encontrados = sorted(pratos_encontrados, key=lambda x: x[1], reverse=True)

    return mensagem, pratos_encontrados[:5] if pratos_encontrados else "Nenhum prato encontrado."


In [103]:
# from scipy.spatial.distance import cosine
# import numpy as np
# from difflib import get_close_matches

# def encontrar_pratos_por_cerveja(nome_cerveja, cervejas_df, pratos_df, modelo, atributos_cerveja, todos_sabores, pesos=None):
#     # Converter o nome da cerveja para minúsculas
#     nome_cerveja_lower = nome_cerveja.lower()
    
#     # Buscar os atributos da cerveja na coluna 'estilo' ignorando maiúsculas e minúsculas
#     cerveja = cervejas_df[cervejas_df['estilo'].str.lower().str.contains(nome_cerveja_lower, na=False)]
    
#     if cerveja.empty:
#         estilos_similares = set(get_close_matches(nome_cerveja, cervejas_df['estilo'].tolist(), n=3, cutoff=0.5))
#         if estilos_similares:
#             estilo_similar = list(estilos_similares)[0]
#             mensagem = f"Cerveja similar encontrada: '{estilo_similar}'."
#             cerveja_similar = cervejas_df[cervejas_df['estilo'].str.lower() == estilo_similar.lower()]
#             atributos_cerveja_valores = cerveja_similar[atributos_cerveja].values
#         else:
#             return f"Cerveja '{nome_cerveja}' não encontrada e nenhum estilo similar disponível.", []
#     else:
#         mensagem = f"Cerveja encontrada: '{nome_cerveja}'."
#         atributos_cerveja_valores = cerveja[atributos_cerveja].values
    
#     # Prever os atributos dos pratos usando o modelo
#     atributos_previstos = modelo.predict(atributos_cerveja_valores)
    
#     if atributos_previstos.shape[0] == 0:
#         return mensagem, "Não foi possível prever os atributos para a cerveja."

#     atributos_previstos_medios = np.mean(atributos_previstos, axis=0).reshape(1, -1)
#     atributos_pratos = pratos_df[todos_sabores].values
    
#     if atributos_pratos.size == 0:
#         return mensagem, "A base de pratos está vazia."

#     pratos_encontrados = []

#     # Cálculo da similaridade com média ponderada
#     for i, atributos_prato in enumerate(atributos_pratos):
#         similaridades_individuais = 1 - np.abs(atributos_prato - atributos_previstos_medios[0])  # 1 - diferença para obter similaridade

#         # Normalizar similaridades para percentuais
#         similaridades_percentuais = similaridades_individuais * 100

#         # Média ponderada
#         if pesos is None:
#             pesos = np.ones(len(todos_sabores)) / len(todos_sabores)  # Pesos iguais caso não sejam especificados
#         media_ponderada = np.average(similaridades_percentuais, weights=pesos)

#         prato = pratos_df.iloc[i]['pratos_allrecipes']
#         pratos_encontrados.append((prato, media_ponderada))

#     # Ordenar os pratos pelo valor mais alto de média ponderada
#     pratos_encontrados = sorted(pratos_encontrados, key=lambda x: x[1], reverse=True)

#     return mensagem, pratos_encontrados[:5] if pratos_encontrados else "Nenhum prato encontrado."


In [76]:
pratos

'jazzy stuffed peppers'

In [229]:
resultado = encontrar_pratos_por_cerveja('Belgian White', cervejas, pratos, modelo_nn, atributos_cerveja, todos_sabores)
if isinstance(resultado, tuple):
    estilo_similar, pratos = resultado
    print(estilo_similar)  # Mostra o estilo similar encontrado
    for prato, porcentagem in pratos:
        print(f"Prato: {prato}: {porcentagem:.2f}%")
else:
    print(resultado)  # Mensagem de erro ou estilo similar não encontrado


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step  
Cerveja similar encontrada: 'Belgian Tripel'.
Prato: pesto sauce: 82.01%
Prato: pate: 67.41%
Prato: avocado lime cheesecake: 66.97%
Prato: juiciest hamburgers ever: 63.68%
Prato: no churn blackberry ice cream: 63.62%


In [70]:
resultado = encontrar_pratos_por_cerveja('IPA', cervejas, pratos, modelo_nn, atributos_cerveja, todos_sabores)
if isinstance(resultado, tuple):
    estilo_similar, pratos_ = resultado
    print(estilo_similar)  # Mostra o estilo similar encontrado
    for prato, porcentagem in pratos_:
        print(f"Prato: {prato}: {porcentagem:.2f}%")
else:
    print(resultado)  # Mensagem de erro ou estilo similar não encontrado


38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step
Cerveja encontrada: 'IPA'.
Prato: peppercorn sauce: 70.27%
Prato: 4 ingredient hamburger casserole: 68.55%
Prato: roast beef and cheddar sliders: 66.50%
Prato: jazzy stuffed peppers: 65.70%
Prato: pate: 62.49%


In [129]:
resultado = encontrar_pratos_por_cerveja('American Pale Ale', cervejas, pratos, modelo_nn, atributos_cerveja, todos_sabores)
if isinstance(resultado, tuple):
    estilo_similar, pratos_ = resultado
    print(estilo_similar)  # Mostra o estilo similar encontrado
    for prato, porcentagem in pratos_:
        print(f"Prato: {prato}: {porcentagem:.2f}%")
else:
    print(resultado)  # Mensagem de erro ou estilo similar não encontrado


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Cerveja encontrada: 'American Pale Ale'.
Prato: jazzy stuffed peppers: 90.28%
Prato: peppercorn sauce: 89.99%
Prato: sun dried tomato pesto no nuts: 89.10%
Prato: 4 ingredient hamburger casserole: 88.69%
Prato: pate: 88.45%


In [43]:
resultado = encontrar_pratos_por_cerveja('WITBIER', cervejas, pratos, modelo_nn, atributos_cerveja, todos_sabores)
if isinstance(resultado, tuple):
    estilo_similar, pratos_ = resultado
    print(estilo_similar)  # Mostra o estilo similar encontrado
    for prato, porcentagem in pratos_:
        print(f"Prato: {prato}: {porcentagem:.2f}%")
else:
    print(resultado)  # Mensagem de erro ou estilo similar não encontrado


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Cerveja encontrada: 'WITBIER'.
Prato: nats shrimp and veggie stuffed zucchini: 93.36%
Prato: shrimp scampi: 92.34%
Prato: stuffed bell peppers: 90.74%
Prato: garlic shrimp linguine: 90.47%
Prato: orzo stuffed peppers: 90.26%


In [60]:
resultado = encontrar_pratos_por_cerveja('IPA', cervejas, pratos, modelo_nn, atributos_cerveja, todos_sabores)
if isinstance(resultado, tuple):
    estilo_similar, pratos_ = resultado
    print(estilo_similar)  # Mostra o estilo similar encontrado
    for prato, porcentagem in pratos_:
        print(f"Prato: {prato}: {porcentagem:.2f}%")
else:
    print(resultado)  # Mensagem de erro ou estilo similar não encontrado


38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Cerveja encontrada: 'IPA'.
Prato: stuffed bell peppers: 94.44%
Prato: orzo stuffed peppers: 93.63%
Prato: nats shrimp and veggie stuffed zucchini: 90.72%
Prato: italian stuffed cubanelle peppers: 90.25%
Prato: watermelon feta salad: 89.46%


In [48]:
resultado = encontrar_pratos_por_cerveja('Imperial Stout', cervejas, pratos, modelo_nn, atributos_cerveja, todos_sabores)
if isinstance(resultado, tuple):
    estilo_similar, pratos_ = resultado
    print(estilo_similar)  # Mostra o estilo similar encontrado
    for prato, porcentagem in pratos_:
        print(f"Prato: {prato}: {porcentagem:.2f}%")
else:
    print(resultado)  # Mensagem de erro ou estilo similar não encontrado


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Cerveja encontrada: 'Imperial Stout'.
Prato: classic strawberry crumble: 94.13%
Prato: strawberry rhubarb cobbler: 93.89%
Prato: cheesecake in a jar: 93.86%
Prato: southern style strawberry cobbler: 92.75%
Prato: low sugar strawberry rhubarb crunch: 90.50%


In [65]:
resultado = encontrar_pratos_por_cerveja('IPA', cervejas, pratos, modelo_nn, atributos_cerveja, todos_sabores)
if isinstance(resultado, tuple):
    estilo_similar, pratos_ = resultado
    print(estilo_similar)  # Mostra o estilo similar encontrado
    for prato, porcentagem in pratos_:
        print(f"Prato: {prato}: {porcentagem:.2f}%")
else:
    print(resultado)  # Mensagem de erro ou estilo similar não encontrado


38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Cerveja encontrada: 'IPA'.
Prato: peppercorn sauce: 90.70%
Prato: jazzy stuffed peppers: 90.31%
Prato: 4 ingredient hamburger casserole: 90.01%
Prato: roast beef and cheddar sliders: 89.09%
Prato: sun dried tomato pesto no nuts: 89.03%


In [67]:
resultado = encontrar_pratos_por_cerveja('Imperial Stout', cervejas, pratos, modelo_nn, atributos_cerveja, todos_sabores)
if isinstance(resultado, tuple):
    estilo_similar, pratos_ = resultado
    print(estilo_similar)  # Mostra o estilo similar encontrado
    for prato, porcentagem in pratos_:
        print(f"Prato: {prato}: {porcentagem:.2f}%")
else:
    print(resultado)  # Mensagem de erro ou estilo similar não encontrado


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Cerveja encontrada: 'Imperial Stout'.
Prato: strawberry rhubarb cobbler: 91.57%
Prato: best strawberry daiquiri: 91.56%
Prato: frozen strawberry lemonade: 91.55%
Prato: air fryer strawberry crisp: 91.29%
Prato: strawberry acai lemonade: 91.19%


In [141]:
resultado = encontrar_pratos_por_cerveja('IPA', cervejas, pratos, modelo_nn, atributos_cerveja, todos_sabores)

if isinstance(resultado, tuple):
    mensagem, pratos_ = resultado
    print(mensagem)  # Exibe a mensagem (se a cerveja foi encontrada ou se algum estilo similar foi encontrado)
    for prato, similaridade_media in pratos_:
        print(f"Prato: {prato} | Similaridade: {similaridade_media:.2f}%")
else:
    print(resultado)  # Exibe a mensagem de erro ou estilo similar não encontrado


38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Cerveja encontrada: 'IPA'.
Prato: peppercorn sauce | Similaridade: 90.70%
Prato: jazzy stuffed peppers | Similaridade: 90.31%
Prato: 4 ingredient hamburger casserole | Similaridade: 90.01%
Prato: roast beef and cheddar sliders | Similaridade: 89.09%
Prato: sun dried tomato pesto no nuts | Similaridade: 89.03%


In [135]:
resultado = encontrar_pratos_por_cerveja('IMPERIAL STOUT', cervejas, pratos, modelo_nn, atributos_cerveja, todos_sabores)

if isinstance(resultado, tuple):
    mensagem, pratos_ = resultado
    print(mensagem)  # Exibe a mensagem (se a cerveja foi encontrada ou se algum estilo similar foi encontrado)
    for prato, similaridade_media in pratos_:
        print(f"Prato: {prato} | Similaridade: {similaridade_media:.2f}%")
else:
    print(resultado)  # Exibe a mensagem de erro ou estilo similar não encontrado


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Cerveja encontrada: 'IMPERIAL STOUT'.
Prato: strawberry rhubarb cobbler | Similaridade: 91.57%
Prato: best strawberry daiquiri | Similaridade: 91.56%
Prato: frozen strawberry lemonade | Similaridade: 91.55%
Prato: air fryer strawberry crisp | Similaridade: 91.29%
Prato: strawberry acai lemonade | Similaridade: 91.19%


In [137]:
resultado = encontrar_pratos_por_cerveja('WITBIER', cervejas, pratos, modelo_nn, atributos_cerveja, todos_sabores)

if isinstance(resultado, tuple):
    mensagem, pratos_ = resultado
    print(mensagem)  # Exibe a mensagem (se a cerveja foi encontrada ou se algum estilo similar foi encontrado)
    for prato, similaridade_media in pratos_:
        print(f"Prato: {prato} | Similaridade: {similaridade_media:.2f}%")
else:
    print(resultado)  # Exibe a mensagem de erro ou estilo similar não encontrado


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Cerveja encontrada: 'WITBIER'.
Prato: sun dried tomato pesto no nuts | Similaridade: 86.41%
Prato: shrimp scampi | Similaridade: 86.00%
Prato: watermelon feta salad | Similaridade: 85.58%
Prato: chicken liver pate | Similaridade: 85.01%
Prato: cheesecake in a jar | Similaridade: 84.81%


In [184]:
# Lista de cervejas para previsão
cervejas_lista = ['IPA', 'munich dunkle', 'english porter', 'witbier']

# Loop para chamar a função para cada cerveja na lista
for nome_cerveja in cervejas_lista:
    resultado = encontrar_pratos_por_cerveja(nome_cerveja, cervejas, pratos, modelo_nn, atributos_cerveja, todos_sabores)

    if isinstance(resultado, tuple):
        mensagem, pratos_ = resultado
        print(mensagem)  # Exibe a mensagem (se a cerveja foi encontrada ou se algum estilo similar foi encontrado)
        for prato, similaridade_media in pratos_:
            print(f"Prato: {prato} | Similaridade: {similaridade_media:.2f}%")
    else:
        print(resultado)  # Exibe a mensagem de erro ou estilo similar não encontrado


38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Cerveja encontrada: 'IPA'.
Prato: peppercorn sauce | Similaridade: 90.70%
Prato: jazzy stuffed peppers | Similaridade: 90.31%
Prato: 4 ingredient hamburger casserole | Similaridade: 90.01%
Prato: roast beef and cheddar sliders | Similaridade: 89.09%
Prato: sun dried tomato pesto no nuts | Similaridade: 89.03%
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Cerveja similar encontrada: 'Munich Dunkel'.
Prato: cheese fondue | Similaridade: 88.22%
Prato: roast beef and cheddar sliders | Similaridade: 87.78%
Prato: chicken liver pate | Similaridade: 87.05%
Prato: pepperoni pizza dip with cream cheese | Similaridade: 86.80%
Prato: jazzy stuffed peppers | Similaridade: 86.34%
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Cerveja encontrada: 'english porter'.
Prato: 4 ingredient hamburger casserole | Similaridade: 91.72%
Prato: peppercorn sauce | Similaridade: 90.54%
Prato: roast beef and cheddar sliders | Similaridade: 89.17%
Prato: sweet chocolate caramel squares | Simi